# Training & Evaluation
* In this notebook we are going to 2 kinds of object detection models,
    * An Object Detection CNN from scratch, the hypothesis is that since the dataset is simple, we won't need an expert model for object detection. A simple CNN should be able to give us a descent accuracy.
    * Object detection using transfer learning - This is going to more of a hands on experience of transfer learning. 

## Import Libraries

In [1]:
## import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt

2025-07-22 12:02:30.110568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753210950.125902   92819 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753210950.130536   92819 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753210950.143221   92819 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753210950.143244   92819 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753210950.143245   92819 computation_placer.cc:177] computation placer alr

In [2]:
## validate tensorflow 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Constants

In [3]:
data_dir = Path("..","data")
models_dir = Path("..","models")

## Import Scripts

In [4]:
import os
import sys
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import data_generator

## logic to auto reload scripts without restarting the kernel
%load_ext autoreload
%autoreload 2

## Training Object Detection CNN

### Step 1: Import Data

In [5]:
data = pd.read_csv(Path(data_dir,"raw","raw_mnist_data.csv"))

### Step 2: Split Data

In [6]:
raw_images = data.drop(columns=["class"])
raw_labels = data["class"]
raw_images.shape, raw_labels.shape

((70000, 784), (70000,))

### Step 3: Initialize Map Pipeline

In [11]:
X_tensor = tf.convert_to_tensor(raw_images, dtype=tf.float32)
X_tensor = tf.reshape(X_tensor,shape=(-1,28,28,1))
y_tensor = tf.convert_to_tensor(raw_labels, dtype=tf.float32)


raw_dataset = tf.data.Dataset.from_tensor_slices((X_tensor,y_tensor))

processed_dataset = raw_dataset.map(lambda X,y: tf.numpy_function(data_generator.generate_training_example, inp=[X,y],Tout=(tf.float32,tf.float32)), num_parallel_calls=15);

### Step 4: Initialize the Model

In [ ]:
model = tf.keras.Sequential([

    tf.keras.layers.Input(shape=(100,100,1)),
    tf.keras.layers.Rescaling(scale=1./255),

    ## starting with a larger filter since we are dealing with 100x100x1 image
    tf.keras.layers.Conv2D(filters=8, kernel_size=5, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=5, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    ## rest of the layers are same as our original mnist classifier
    tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    ## finaly layers to output 6x6x45 grid of predictions
    tf.keras.layers.Conv2D(filters=45, kernel_size=1, padding='same', activation='linear'),

])

In [21]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 100, 100, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 100, 100, 8)    │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 100, 100, 8)    │         1,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 50, 50, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 50, 50, 8)      │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 50, 50, 8)      │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 25, 25, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 25, 25, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 25, 25, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 12, 12, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 12, 12, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 6, 6, 45)       │         1,485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,845 (85.33 KB)

 Trainable params: 21,845 (85.33 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])